# 文件与IO

## 读写文本数据

In [1]:
with open('data/somefile.txt', 'wt') as f:
    f.write('a\nb\nc')
with open('data/somefile.txt', 'rt') as f:
    data = f.read()
    print(data)

a
b
c


## 打印输出至文件中

In [2]:
with open('data/somefile.txt', 'wt') as f:
    print('Hello world!', file=f)
with open('data/somefile.txt', 'rt') as f:
    data = f.read()
    print(data)

Hello world!



## 使用其他分隔符或行终止符打印

In [3]:
print('ACME', 50, 91.5)
print('ACME', 50, 91.5, sep=',')
print('ACME', 50, 91.5, sep=',', end='!!\n')

ACME 50 91.5
ACME,50,91.5
ACME,50,91.5!!


In [4]:
for i in range(0, 10):
    print(i, end=' ')

0 1 2 3 4 5 6 7 8 9

## 读写字节数据

In [5]:
with open('data/somefile.bin', 'wb') as f:
    f.write(b'Hello world!')
with open('data/somefile.bin', 'rb') as f:
    data = f.read()
    print(data)

b'Hello world!'


## 文件不存在才能写入

In [6]:
with open('data/somefile.txt', 'xt') as f:
    f.write('1') # 会报错，FileExistsError 

FileExistsError: [Errno 17] File exists: 'data/somefile.txt'

## 字符串的I/O操作

In [7]:
import io
s = io.StringIO()
s.write('Hello world!\n')
print('This is a test', file=s)
s.getvalue()

'Hello world!\nThis is a test\n'

In [8]:
s = io.BytesIO()
s.write(b'data')
s.getvalue()

b'data'

## 读写压缩文件

In [9]:
text = 'Hello world!'

import gzip
with gzip.open('data/somefile.gz', 'wt') as f:
    f.write(text)

import bz2
with bz2.open('data/somefile.bz2', 'wt') as f:
    f.write(text)

## 固定大小记录的文件迭代

In [10]:
from functools import partial

RECORD_SIZE = 4

with open('data/somefile.bin', 'rb') as f:
    record = iter(partial(f.read, RECORD_SIZE), b'')
    for r in record:
        print(r)

b'Hell'
b'o wo'
b'rld!'


## 读取二进制数据到可变缓冲区中

In [11]:
import os.path

def read_into_buffer(filename):
    buf = bytearray(os.path.getsize(filename))
    with open(filename, 'rb') as f:
        f.readinto(buf)
    return buf

buf = read_into_buffer('data/somefile.bin')
buf

bytearray(b'Hello world!')

## 内存映射的二进制文件

In [12]:
import os
import mmap

def memo_map(filaname, access=mmap.ACCESS_WRITE):
    size = os.path.getsize(filaname)
    fd = os.open(filaname, os.O_RDWR)
    return mmap.mmap(fd, size, access=access)

size = 100
with open('data/file', 'wb') as f:
    f.seek(size-1)
    f.write(b'\x00')
m = memo_map('data/file')
print(len(m))
print(m[0:9])
m.close()

100
b'\x00\x00\x00\x00\x00\x00\x00\x00\x00'


## 文件路径名的操作

In [13]:
import os

path = 'data/somefile.txt'
print(os.path.basename(path))
print(os.path.dirname(path))
print(os.path.join('tmp', 'data', os.path.basename(path)))


somefile.txt
data
tmp\data\somefile.txt


## 测试文件是否存在

In [14]:
import os
print(os.path.exists('data/somefile.bin'))
print(os.path.exists('data/somefile'))
print(os.path.isfile('data/somefile.bin'))
print(os.path.isdir('data'))


True
False
True
True


## 获取文件夹中的文件列表

In [15]:
import os.path

names = [name for name in os.listdir('data') 
            if os.path.isfile(os.path.join('data', name))]
print(names)

dirs = [d for d in os.listdir('./') 
            if os.path.isdir(os.path.join('./', d)) 
            and not d.startswith('.')]
print(dirs)

['file', 'somefile.bin', 'somefile.bz2', 'somefile.gz', 'somefile.txt']
['data']


In [17]:
import glob
files = glob.glob('data/*.bin')
print(files)

from fnmatch import fnmatch
files = [name for name in os.listdir('data') if fnmatch(name, '*.bin')]
print(files)

['data\\somefile.bin']
['somefile.bin']


## 忽略文件名编码

In [18]:
print(sys.getfilesystemencoding())


utf-8


In [1]:
with open('data/jalape\xf1o.txt', 'w') as f:
    f.write('Spicy!')
import os
print(os.listdir('data'))

['file', 'jalapeño.txt', 'somefile.bin', 'somefile.bz2', 'somefile.gz', 'somefile.txt']


## 打印不合法的文件名

In [ ]:
def bad_filename(filename):
    return repr(filename)[1:-1]

try:
    print(filename)
except UnicodeEncodeError:
    print(bad_filename(filename))

## 增加或改变已打开文件的编码

如果你想给一个以二进制模式打开的文件添加 Unicode 编码/解码方式，可以使用io.TextIOWrapper() 对象包装它。

## 将字节写入文本文件

将字节数据直接写入文件的缓冲区即可

## 将文件描述符包装成文件对象

In [2]:
# 打开文件描述符
import os
fd = os.open('data/somefile.txt', os.O_WRONLY | os.O_CREAT)
# 包装成文件对象
f = open(fd, 'wt')
f.write('hello world\n')
f.close()

## 创建临时文件和文件夹

In [3]:
from tempfile import TemporaryFile
with TemporaryFile('w+t') as f:
    # Read/write to the file
    f.write('Hello World\n')
    f.write('Testing\n')
    # Seek back to beginning and read the data
    f.seek(0)
    data = f.read()
    print(data)

Hello World
Testing



## 与串行端口的数据通信

尽管可以通过使用 Python 内置的 I/O 模块来完成这个任务，但对于串行通信,最好的选择是使用 pySerial 包 。

## 序列化 Python 对象

In [6]:
import pickle
data = [1, 2, 3, 4]
f = open('data/somefile', 'wb')
pickle.dump(data, f)

In [7]:
f = open('data/somefile', 'rb')
data = pickle.load(f)
data

[1, 2, 3, 4]